# Assignment: 1NN против RandomForest


В этом задании будет использоваться датасет `digits` из `sklearn.datasets`. Оставьте _последние_ 25% объектов для контроля качества, разделив `X` и `y` на `X_train`, `y_train` и `X_test`, `y_test`.

In [6]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами `1NN` с `RandomForestClassifier` из `sklearn` на 1000 деревьях.

## Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

In [33]:
from sklearn.metrics import accuracy_score
from scipy.spatial.distance import euclidean
from tqdm import tqdm

In [32]:
def one_nn_predict(X_test, X_train, y_train):
    container = list()
    for x in tqdm(X_test):
        min_distance = np.inf
        min_distance_class = 0
        for idx, vector in enumerate(X_train):
            distance = euclidean(vector, x)
            if distance < min_distance:
                min_distance = distance
                min_distance_class = y_train[idx]
        container.append(min_distance_class)
    return container

In [36]:
prediction = one_nn_predict(X_test, X_train, y_train)

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:11<00:00, 38.58it/s]


In [37]:
print (accuracy_score(y_test, prediction))

0.9911111111111112


Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка $N log N$ сравнений (более строго говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. 

За выбор метода поиска ближайших соседей в `KNeigborsClassifier` из `sklearn` отвечает параметр `algorithm` — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных `ball tree` и `kd tree`.

In [38]:
from sklearn.neighbors import KNeighborsClassifier

In [39]:
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)
print (accuracy_score(y_test, model.predict(X_test)))

0.9911111111111112


Доля ошибок, допускаемых 1NN на тестовой выборке, — **ответ в задании 1**.

## Задание 2

Теперь обучите на обучающей выборке `RandomForestClassifier(n_estimators=1000)` из `sklearn`. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — **ответ в задании 2**. 

In [40]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
print (accuracy_score(y_test, model.predict(X_test)))

0.9755555555555555


Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — `1NN`. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.